# Problem set 3: Loading and structuring data from Denmark Statistics

In [15]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
import pandas as pd
import ipywidgets as widgets

# Tasks

## Create a pandas DataFrame

Modify the code below such that *income* and *consumption* are variables in the *dt* DataFrame.

In [16]:
np.random.seed(1999)
 
N = 100
mydata = {}
mydata['id'] = range(N)
income = np.exp(np.random.normal(size=N))
consumption = np.sqrt(income)

mydata['income'] = income
mydata['consumption'] = consumption

dt = pd.DataFrame(mydata)
dt.head()

,id,income,consumption
0,0,0.727981,0.853218
1,1,1.997831,1.413447
2,2,0.276823,0.526140
3,3,1.481931,1.217346
4,4,1.235904,1.111712


**Answer:** see A01.py

## Create new variable

1) Add a new variable *ratio* which is the ratio of consumption to income.

In [17]:
dt['ratio'] = dt.consumption / dt.income
dt.head()

,id,income,consumption,ratio
0,0,0.727981,0.853218,1.172033
1,1,1.997831,1.413447,0.707490
2,2,0.276823,0.526140,1.900636
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513


**Answer:** See A02.py

## Summary statistics

Produce summary statistics using `.describe()`.

In [18]:
# write your code here
dt.describe()

,id,income,consumption,ratio
count,100.000000,100.000000,100.000000,100.000000
mean,49.500000,1.415547,1.087844,1.117517
std,29.011492,1.322203,0.484238,0.525452
min,0.000000,0.108402,0.329245,0.351134
25%,24.750000,0.529323,0.727545,0.752310
50%,49.500000,0.981178,0.990533,1.009580
75%,74.250000,1.768211,1.329572,1.374491
max,99.000000,8.110612,2.847914,3.037255


**Answer:** See A03.py

## Indexing

Select everybody with an income above 1.

In [19]:
I = dt['income'] > 1  ## My favorite approach: 1. Create indicator for the condition
dt.loc[I,:].head()    ## 2. Apply the indicator to the data frame

,id,income,consumption,ratio
1,1,1.997831,1.413447,0.707490
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513
6,6,2.574032,1.604379,0.623294
7,7,2.475478,1.573365,0.635580


**Answer:** See A04.py

Select everybody with an income *above* 1 and a ratio *above* 0.7.

In [20]:
I = (dt.income > 1) & (dt.ratio > 0.7)
dt.loc[I,:].head()

,id,income,consumption,ratio
1,1,1.997831,1.413447,0.707490
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513
11,11,2.031708,1.425380,0.701567
18,18,1.280235,1.131475,0.883802


**Answer:** See A05.py

Set consumption equal to 0.5 if income is less than 0.5.

In [21]:
I = dt['income'] < 0.5
dt.loc[I, 'consumption'] = 0.5
dt['consumption'].mean() # <- compare with answer

1.075479712048503

**Answer:**  See A06.py

Set consumption equal to income if income is less than 0.5.

In [22]:
I = dt.income < 0.5
dt.loc[I, 'consumption'] = dt.loc[I, 'income']
dt['consumption'].mean() # <- compare with answer

1.0337728690050054

**Answer:** See A07.py

## Dropping

Drop the *ratio* variable and all rows with an income above 1.5. After this, also drop the first 5 rows.

In [23]:
print(f'before: {dt.shape[0]} observations, {dt.shape[1]} variables')

dt.drop('ratio', axis=1, inplace=True)  ## drop the ratio variable - the inplace keyword means that this is done by modifying the data frame directly rather than creating a copy of the data frame with the variable dropped

I = dt.income > 1.5
dt.drop(dt.loc[I].index, inplace=True)  ## Use .index to drop the rows where the condition is true

dt.drop(dt.iloc[0:5].index, inplace=True)   ## Use .iloc[0:5] to drop the first 5 rows - remember, .iloc uses numerical indexing

print(f'after: {dt.shape[0]} observations, {dt.shape[1]} variables')

before: 100 observations, 4 variables
after: 65 observations, 3 variables


**Answer:** see A08.py

## Renaming

Rename *consumption* to *cons* and *income* to *inc*.

In [24]:
dt.rename(columns={'income': 'inc', 'consumption': 'cons'}, inplace=True) ## We can rename columns using the .rename method and a dictionary of old and new names
dt.head()

,id,inc,cons
8,8,0.582074,0.762938
10,10,0.932044,0.965424
12,12,0.356952,0.356952
13,13,0.379825,0.379825
16,16,0.700896,0.837195


**Answer:** see A09.py

## Functions

Correct the wrong lines such that `assets_1 = assets_2 = assets_3 = assets_4`.

In [25]:
def assets_row_by_row(x,R,Y):
    return R*(x['inc']-x['cons'])+Y  ## x is a row of the data frame, so this function applies the formula to each row
    
def assets_all_at_once(income,consumption,R,Y):
    return R*(income-consumption)+Y  ## This function applies the formula to the entire array at once (pd.Series and np.array objects are designed to work with element-wise operations)

def assets_adj(assets,R,Y):
    assets *= R
    assets += Y                     ## This function modifies the pandas series in place, so it does not need to return anything

R = 1.2 # return rate
Y = 1 # income
try:
    dt['assets_1'] = R*(dt['inc']-dt['cons'])+Y
    dt['assets_2'] = dt.apply(assets_row_by_row,axis=1,args=(R,Y))
    dt['assets_3'] = assets_all_at_once(dt['inc'].values,dt['cons'].values,R,Y)
    dt['assets_4'] = dt['inc']-dt['cons']
    assets_adj(dt['assets_4'],R,Y)
except:
    print('failed')
dt.head()    

,id,inc,cons,assets_1,assets_2,assets_3,assets_4
8,8,0.582074,0.762938,0.782963,0.782963,0.782963,0.782963
10,10,0.932044,0.965424,0.959943,0.959943,0.959943,0.959943
12,12,0.356952,0.356952,1.000000,1.000000,1.000000,1.000000
13,13,0.379825,0.379825,1.000000,1.000000,1.000000,1.000000
16,16,0.700896,0.837195,0.836441,0.836441,0.836441,0.836441


**Answer:** see A10.py

# Problem

Load the data set in *data/NAH1_pivoted.xlsx* and clean and structure it such that the `plot_timeseries(dataframe)` below can be run and produce an interactive figure. 

In [26]:
def _plot_timeseries(dataframe, variable, years):
    
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    dataframe.loc[:,['year']] = pd.to_numeric(dataframe['year'])
    I = (dataframe['year'] >= years[0]) & (dataframe['year'] <= years[1])
    
    x = dataframe.loc[I,'year']
    y = dataframe.loc[I,variable]
    ax.plot(x,y)
    
    ax.set_xticks(list(range(years[0], years[1] + 1, 5)))    
    
def plot_timeseries(dataframe):
    
    widgets.interact(_plot_timeseries, 
    dataframe = widgets.fixed(dataframe),
    variable = widgets.Dropdown(
        description='variable', 
        options=['Y','C','G','I','X','M'], 
        value='Y'),
    years=widgets.IntRangeSlider(
        description="years",
        min=1966,
        max=2018,
        value=[1980, 2018],
        continuous_update=False,
    )                 
); 

**Hint 1:** You can base your renaming on this dictionary:

In [27]:
rename_dict = {}
rename_dict['P.1 Output'] = 'Y'
rename_dict['P.3 Final consumption expenditure'] = 'C'
rename_dict['P.3 Government consumption expenditure'] = 'G'
rename_dict['P.5 Gross capital formation'] = 'I'
rename_dict['P.6 Export of goods and services'] = 'X'
rename_dict['P.7 Import of goods and services'] = 'M'

**Hint 2:** You code should have the following structure:

In [28]:
# a. load data set
nah1 = pd.read_excel('data/NAH1_pivoted.xlsx', skiprows=2)

# b. rename variables
rename_dict['Unnamed: 0'] = 'year'
rename_dict['Unnamed: 1'] = 'prices'
nah1.rename(columns=rename_dict, inplace=True)

# c. remove rows where Y is nan
I = nah1['Y'].notnull()         ## I indexes the rows where Y is not null
nah1 = nah1.loc[I]              ## keep only when I is True

# d. correct year column data
I = nah1['year'].notnull()       ## Use that the data is sorted with alternately missing and non-missing values - we want to fill out the missing values with the previous non-missing value
J = nah1['year'].isnull()
nah1.loc[J,['year']] = nah1.loc[I,['year']].values

# e. only keep rows with '2010-prices, chained values'
I = nah1['prices'] == '2010-prices, chained values'
J = nah1['prices'] == 'Current prices'
nah1_nom = nah1.loc[J]      ## Saving the df of current prices for later
nah1 = nah1.loc[I]

# f. only keep renamed variables
nah1 = nah1.loc[:,rename_dict.values()]  ## Use the dictionary values to get the new names of the variables
nah1_nom = nah1_nom.loc[:,rename_dict.values()]

# g. interactive plot
plot_timeseries(nah1)

interactive(children=(Dropdown(description='variable', options=('Y', 'C', 'G', 'I', 'X', 'M'), value='Y'), Int…

**Answer:** see A11.py

# Extra problems

## Extend interactive plot

Extend the interactive plot with a choice of *real* vs *nominal*.

In [29]:
## Note that earlier, I saved the variables in nominal prices in a df called nah1_nom
full_nah1 = pd.concat((nah1, nah1_nom), axis=0)

# define plotting function
def _plot_timeseries_new(dataframe, variable, years, prices): #include prices as input in helper function
    
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    dataframe.loc[:,['year']] = pd.to_numeric(dataframe['year'])
    I = (dataframe['year'] >= years[0]) & (dataframe['year'] <= years[1])
    
    I &= dataframe['prices'] == prices # select prices
    
    x = dataframe.loc[I,'year']
    y = dataframe.loc[I,variable]
    ax.plot(x,y)
    
    ax.set_xticks(list(range(years[0], years[1] + 1, 5)))


def plot_timeseries_new(dataframe):

    widgets.interact(_plot_timeseries_new, 
    dataframe = widgets.fixed(value=dataframe),
    prices = widgets.Dropdown(
        description='prices',
        options=['2010-prices, chained values', 'Current prices'],
        value='2010-prices, chained values'), #include prices in dropdown menu
    variable = widgets.Dropdown(
        description='variable', 
        options=['Y','C','G','I','X','M'], 
        value='Y'),
    years=widgets.IntRangeSlider(
        description="years",
        min=1966,
        max=2018,
        value=[1980, 2018],
        continuous_update=False,
    )                 
) 

plot_timeseries_new(full_nah1)

interactive(children=(Dropdown(description='variable', options=('Y', 'C', 'G', 'I', 'X', 'M'), value='Y'), Int…

## New data set

Load data from an Excel or CSV file you have downloaded from e.g. [Statistikbanken.dk](https://www.statistikbanken.dk/). Clean, structure and present the data as you see fit.